<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Data Preprocessing in Python: Data Integration and Reshaping

I will cover some important processes for DFs:
* Appending
* Reshaping
* Merging

# 1. Appending

As the name implies, this process binds DFs into one, that is, one or more DFs will be put below or on top of another DF. Appending can be done when you fulfill these requisites:
1. All the DFs  share the same column names.
2. All the DFs  columns are in the same location.
2. All the DFs  columns have the same data types.

Let's visit this website: https://fundforpeace.org/what-we-do/country-risk-and-fragility-data/

There, you will find several excel files with the _Fragile States Index_ per year. Let's downloan the ones for years 2019-2021 in the folder where this notebook is stored. Then, we can open them:

In [ ]:
import pandas as pd

file2021="fsi-2021.xlsx"
file2020="fsi-2020.xlsx"
file2019="fsi-2019.xlsx"
    

# fetching the tables
fragil2021=pd.read_excel(file2021)
fragil2020=pd.read_excel(file2020)
fragil2019=pd.read_excel(file2019)

The goal is to append all of them. Let's see if they have the same column names with the help of **set**s:

In [ ]:
A=['a','b','c']
B=['a','b','d','c']
C=['a','b','e','g']

# set intersection
set(A) & set(B) & set(C)

Then:

In [ ]:
# these are the columns shared among the DFs
set(fragil2021.columns)&set(fragil2020)&set(fragil2019)

We can create a new DF by appending the DFs that share those columns:

In [ ]:
common=set(fragil2021.columns)&set(fragil2020)&set(fragil2019)

# we are keeping just the 'common' columns:
fragil2021ap=fragil2021.loc[:,common]
fragil2020ap=fragil2020.loc[:,common]
fragil2019ap=fragil2019.loc[:,common]

Since all the DFs were created using the same columns, you just fulfilled the first two requisites.

The last requisite is to make sure they share the same data types. Let's check the data types with the help of **zip**, **set** and **len**:

In [ ]:
A=['a','b','c']
B=['a','b','c']
C=['a','b','d']

# zip will pair elements in the same position:
list(zip(A,B,C))

Combining _len_ and _set_ you have a measure of 'variety'. If variety of a _set_ equals **1**, all the elements in a _set_ are the same:

In [ ]:
len(set([1,1,1,1,1,1,1,1,1,1])), \
len(set([1,2,1])), \
len(set([1,111,11]))

Then, you detect where a group does not have the same elements:

In [ ]:
# check if every zipped group has a variety greater than 1.
[x for x in zip(A,B,C) if len(set(x))>1]

Applying that to our case:

In [ ]:
theZips=zip(fragil2021.dtypes,
            fragil2020.dtypes,
            fragil2019.dtypes)

[x for x in theZips if len(set(x))>1]

From the result above, there is one set of columns that share different data types. Let me modify the previous code to detect the location:

In [ ]:

theZips=zip(fragil2021.dtypes.index,
            fragil2021.dtypes,
            fragil2020.dtypes,
            fragil2019.dtypes)

[x for x in theZips if len(set(x))>2]

The Year is not an integer in the DFs for 2020 and 2019, it is **date** type. 

In [ ]:
fragil2020['Year']

Using the **dt** attribute of date columns in Pandas, we can recover just the year as an integer:

In [ ]:
fragil2020['Year']=fragil2020['Year'].dt.year
fragil2019['Year']=fragil2019['Year'].dt.year

Now it should be fine:

In [ ]:
theZips=zip(fragil2021.dtypes,fragil2020.dtypes,fragil2019.dtypes)
[x for x in theZips if len(set(x))>1]

As we have met al the requisites, let's do the appending!

In [ ]:
fragile=pd.concat([fragil2021,fragil2020,fragil2019])
fragile

You may notice that the column ordering does not look nice, as in general you expect that the columns to the left start with identification of the case rather than measurements; then, let's move 'Country','Year','Total' to the left:

In [ ]:
# this is a trick: setting columns as index
fragile.set_index(['Country','Year','Total'],inplace=True)
fragile

Since I will not use _Rank_, I will get rid of it:

In [ ]:
fragile.drop(columns='Rank',inplace=True)
fragile

Now put the row indexes back:

In [ ]:
fragile.reset_index(inplace=True)
fragile

In [ ]:
fragile.columns=fragile.columns.str.replace(':|\s',"",regex=True)
fragile

In [ ]:
fragile.info()

# 2. Reshaping

Data frames have have different shapes. Let me keep some columns from the last DF so you can notice something:

In [ ]:
fragileLong=fragile.iloc[:,:3]
fragileLong

The unit of analysis is country. Generally, we are used to see the  unit of analysis once in a column, but it is repeated above, as the country will appear for every year of measurement.

Also, pay attention to the **amount of rows**. There are 535 rows, then:

In [ ]:
535/3

That is the amount of rows per data frame, so something went wrong during the appending. The truth is that cleaning and formatting will be needed after complex operations like these.

What is your best guess on what went wrong?

Whatever it is, let me turn our **long** into **wide** shape:

In [ ]:
# wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total', # values to use
               index=['Country'], # unit of analysis
               columns=['Year']) # the values for NEW column
fragileWide

The **wide** shape from a **pivot_table** function looks great, but pay attention:

In [ ]:
fragileWide.info()

As you see above, the country is indeed and index, not a column. This is important if you are planing to export this DF. Also, notice that the column names have a title (_Year_). So in general, you can use this code after the *pivot_table* function:

In [ ]:
fragileWide= fragileWide.reset_index(drop=False).\
             rename_axis(index=None, columns=None)

# result:

fragileWide

I mentioned that something went wring during the appending process. Here, we can discover it:

In [ ]:
# what cells have missing values?
fragileWide[fragileWide.isna().any(axis=1)]

As you can see, even though the data DFs were prepared by the same [organization](https://fundforpeace.org/), the DFs have country names that differ among them. Here we need some **manual** changes: 

In [ ]:
# prepare changes as dict:
changes={"Cabo Verde": "Cape Verde",
"Czechia":"Czech Republic",
"Swaziland":"Eswatini",
"Israel and West Bank":"Israel",
"Kyrgyzstan":"Kyrgyz Republic",
"North Macedonia":"Macedonia",
"Slovakia": "Slovak Republic"}

# make changes using 'replace':
fragileLong.Country.replace(to_replace=changes,inplace=True)

Notice I had to make the changes in the long shape of the DF, so that the wide shape will work fine:

In [ ]:
# wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total',
               index=['Country'],
               columns=['Year']).\
            reset_index(drop=False).\
            rename_axis(index=None, columns=None)
##
fragileWide[fragileWide.isna().any(axis=1)]

You have to be very careful when working with countries, specially when you are including or excluding countries; which may cause you hurting someone else's feelings. 

For instance, here I am just keeping **rows** with no missing values:

In [ ]:
fragileWide.dropna(inplace=True) # axis=1 for columns
fragileWide

Also, notice that the final line above says that the last row index is 178; which means there are 179 rows. To correct that, reset the index:

In [ ]:
fragileWide.reset_index(drop=True, inplace=True)
fragileWide

# 3. Merging

Integrating or merging data sets needs the following considerations:

* Merging is done on two data frames.
* You need a column in each data frame with that share the same exact and unique values. The column names or titles need not be the same.
* The merged table shows by default the mutual coincidences; but you can also request the values not matched, which will help you detect possible extra cleaning.
* Pandas differentiates the **left** from the **right** data frames.

At this stage, let me use two data frames to show how this works:

In [ ]:
demo=pd.read_pickle("https://github.com/EvansDataScience/CTforGA_cleaning/raw/main/demoindex.pkl")
#and
fragile2021=fragil2021ap[fragil2021ap.Year==2021].drop(columns=['Year'])

In [ ]:
fragile2021.info()

Remember the amount of rows of each DF:

In [ ]:
demo.shape,fragile2021.shape

The best scenario would be to create a merged DF with 167 rows, but the actual result is:

In [ ]:
demo.merge(fragile2021).shape

We are some countries far from the best scenario.

Let's use set operations to find what countries are not matching:

In [ ]:
OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyDemo

In [ ]:
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
OnlyFragile

Here, we should try to find the what countries in _OnlyFragile_ may match the ones in _OnlyDemo_. We need to use the **fuzzy merge** approach:

In [ ]:
from thefuzz import process as fz

# look for a country in OnlyDemo and return the most similar
[(fz.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

Above you found the best match. As you see the ones that are wrong have  95% match or less. Let's just filter those:

In [ ]:
[(fz.extractOne(demo, OnlyFragile),demo) 
 for demo in sorted(OnlyDemo) 
 if fz.extractOne(demo, OnlyFragile)[1]>=95]

Once you have good matches, you have to create dictionary like this:

In [ ]:
changesFragile1={fz.extractOne(demo, OnlyFragile)[0]:demo 
                 for demo in sorted(OnlyDemo) 
                 if fz.extractOne(demo, OnlyFragile)[1]>=95}
#dict of matches
changesFragile1

You can use that dict for the replacements:

In [ ]:
fragile2021.Country.replace(to_replace=changesFragile1,inplace=True)

Now the countries in fragile2021 have more matches. 

This process can be done a few more times, and you can recover more rows for the merging process. Let's see:

In [ ]:
# second try
OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
[(fz.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

In [ ]:
# second dict of changes
changesFragile2={fz.extractOne(demo, OnlyFragile)[0]:demo 
                 for demo in sorted(OnlyDemo) 
                 if fz.extractOne(demo, OnlyFragile)[1]>=80}
#dict of matches
changesFragile2

In [ ]:
# make the changes
fragile2021.Country.replace(to_replace=changesFragile2,inplace=True)

In [ ]:
# third try
OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
[(fz.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

In [ ]:
# third dict of changes
changesFragile3={fz.extractOne(demo, OnlyFragile)[0]:demo 
                 for demo in sorted(OnlyDemo) 
                 if fz.extractOne(demo, OnlyFragile)[1]>=64}
#dict of matches
changesFragile3

In [ ]:
# make changes
fragile2021.Country.replace(to_replace=changesFragile3,inplace=True)

In [ ]:
# fourth try

OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
[(fz.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

The fourth attempt did not offer good results. Those two countries will not be able to be matched. Let's retry the merge:

In [ ]:
demo.merge(fragile2021).shape

Then:

In [ ]:
demo_fragile=demo.merge(fragile2021)

In [ ]:
#checking:
demo_fragile.info()

It all look great so far. However, once you think you have the data ready, you should see the basic statistical summary of the data:

In [ ]:
demo_fragile.describe()

A boxplot may also be helpful:

In [ ]:
demo_fragile.plot(kind='box', rot=90)

Here, let's pay attention to the lowest and highest values. In this case, the **Total** has a range of values different than the rest. Let's make sure this columns shares the same range:

In [ ]:
from sklearn import preprocessing

# prepare the process
scaler = preprocessing.MinMaxScaler(feature_range=(0, 10))

# apply process
arrayTotal = scaler.fit_transform(demo_fragile[['Total']])

# result
arrayTotal

You just got:

In [ ]:
type(arrayTotal)

Let me use that array to replace my values:

In [ ]:
demo_fragile['Total']=arrayTotal

Now, these are my data values:

In [ ]:
demo_fragile.plot(kind='box', rot=90)

____


# <font color="red">Exporting file</font>

The current *demo_fragile* data frame is clean and formatted. It is time to send it to a format that will keep all our work for future use:

#### For future use in Python:

In [ ]:
demo_fragile.to_pickle("demo_fragile.pkl")
# you will need: DF=pd.read_pickle("demo_fragile.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://..../demo_fragile.pkl"),compression=None)

#### For future  use in R:

In [ ]:
#try the following before starting Python:
#export LD_LIBRARY_PATH="$(python -m rpy2.situation LD_LIBRARY_PATH)":${LD_LIBRARY_PATH}

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(demo_fragile,file="demo_fragile.RDS")


#In R, you call it with: DF = readRDS("demo_fragile.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://..../demo_fragile.RDS")